In [ ]:
import time
import pandas as pd
import numpy as np
from sklearn import tree

start = time.time()

def initializeData():
    trainFrame = pd.read_json('/Users/nlehr24/Downloads/RTE/train.jsonl', lines=True)

    testFrame = pd.read_json('/Users/nlehr24/Downloads/RTE/test.jsonl', lines=True)

    valFrame = pd.read_json('/Users/nlehr24/Downloads/RTE/val.jsonl', lines=True)

    return trainFrame, testFrame, valFrame

train, test, val = initializeData()
# print(train.head())
# print(test.head())
# print(val.head())
train = np.array(train)
test = np.array(test)
val = np.array(val)

Y_train = [a[-2] for a in train]
X_train = [a[0:2] for a in train]

X_train = np.array(X_train)
Y_train = np.array(Y_train)

# validation data is the same format as the training. Testing data does not contain entailment dictation
Y_val = [a[-2] for a in val]
X_val = [a[0:2] for a in val]

X_val = np.array(X_val)
Y_val = np.array(Y_val)

# difficult to turn words into numbers because of all the layers and variable shapes
#2/20 3:04 process to ints, then find max sentence length, then fill in non-characters in other sentences with -1
# as opposed to image processing, here there is a variable number of inputs
def maxxx(threeDarray):
    lengths = []
    for i in threeDarray:
        for j in i:
            lengths.append(len(list(j)))

    return max(lengths)

# combining ver
def maxx(twoDarray):
    
    lengths = []
    for s in twoDarray:
        lengths.append(len(str(s)))

    return max(lengths)

# def translateToInts(stringSetArray):
#     translated = np.zeros_like(stringSetArray)
    
#     maxLength = maxxx(stringSetArray)
    
#     # for each set of sentences,
#     for i in range(stringSetArray.shape[0]):

#         # for each sentence in each set,
#         for j in range(stringSetArray.shape[1]):

#             sentence = np.zeros([])

#             for k in range(maxLength):
#                 try:
#                     newChar = ord(stringSetArray[i][j][k])
#                 except Exception as e:
#                     newChar = -1
#                 sentence = np.append(sentence, newChar)
    
#             translated[i][j] = sentence

#     return translated

def combine(stringSetArray):
    new = list()
    for i in range(0, stringSetArray.shape[0]):
        # making doubles because not accounting for the changing in length
        stringSetArray[i] = stringSetArray[i][0] + stringSetArray[i][1]
        
        # 2/21 9:44 trying to delete the double made by the indexing reference the uncombined string set array
        stringSetArray[i] = np.delete(stringSetArray[i], 1, axis=0)
        
        new.append(np.delete(stringSetArray[i], 1, axis=0))

    return np.array(new)

maxLength = maxx(combine(X_train))  # making global so that the model can fit the validation data. This assumes maxx(X_train) > maxx(X_val)
# combining ver
def translateToInts(stringArray):
    translated = list()
    
    for i in range(stringArray.shape[0]):
        sentence = []
        string = stringArray[i]

        for j in range(maxLength):
            # not excluding any characters. Figuring the model can just learn around the []s and 's
            
            try:
                newChar = ord(str(string)[j])
            except Exception as e:
                newChar = 0
            sentence.append(newChar)
        # 91: [
        # 39: '
        # 46: .
        translated.append(sentence)
      
    return np.array(translated)

DTC = tree.DecisionTreeClassifier()

print('X initial shape:', X_train.shape, X_val.shape)
X_train = combine(X_train)
X_val = combine(X_val)
print('X shape after combining:', X_train.shape, X_val.shape)
X_train = translateToInts(X_train)
X_val = translateToInts(X_val)
print('X shape after translatingToInts:', X_train.shape, X_val.shape)

# binarizing Y; if not not entailment save as True. Boolean to int is by default 0 (False) or 1 (True)
Y_train = np.array(['not' not in v for v in Y_train], dtype='int')
Y_val = np.array(['not' not in v for v in Y_val], dtype='int')


print(X_train)
print(Y_train)
DTC.fit(X_train, Y_train)


DTCpredictions = DTC.predict(X_val)
# DTRpredictions = DTR.predict(X_val) tried using a DecisionTreeRegressor, but did no better than the DecisionTreeClassifier

def translateBackToStr(a):
    translated = list()
    for sentence in a:
        new = ''
        for int in sentence:
            if int != -1:
                new += chr(int)
        
        translated.append(new)
    return np.array(translated)

X_val = translateBackToStr(X_val)

DTCcorrect = 0
for i in range(X_val.shape[0]):
    if Y_val[i] == DTCpredictions[i]:
        DTCcorrect += 1

print(f'DTC accuracy: {round(((DTCcorrect / X_val.shape[0]) * 100), 2)}%')

def tryNB(X_train, X_val):
    if X_val.shape[-1] != X_train.shape[-1]:
        X_val = translateToInts(X_val)
        
    from sklearn.naive_bayes import MultinomialNB
    nb = MultinomialNB()
    
    nb.fit(X_train, Y_train)
    
    from sklearn import metrics
    
    # make class predictions for X_test_dtm
    y_pred_class = nb.predict(X_val)
    
    # calculate accuracy of class predictions
    print("=======Accuracy Score===========")
    print(metrics.accuracy_score(Y_val, y_pred_class))

tryNB(X_train, X_val)

print('Time taken:', round(time.time() - start, 2))
